In [3]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
import skimage.io as io

import cv2

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
%load_ext blackcellmagic

## Image I/O and formatting workflows

This notebook will first demonstrate some workflows for reading and writing images to different formats using `cv2` (for fast image processing), `skimage.io` (convenient image I/O), and `h5py` (for data storage in HDF5 format). 

I'll then use these packages and their dependencies to draw and save ROIs and identify the brightest cell in a frame.

In [2]:
data_dir = "E:/transceiver_project/20200715_time_lapse_propagation/time_lapse_20x/"

First, I'll read in a series of 5 consecutive brightfield images from a time-lapse experiment and convert them to an `ImageCollection` object.

In [6]:
# Get test image filenames
vid = "0test"
ims_f = glob(os.path.join(data_dir, vid, "*/*_CH3.tiff"))

# Convert to image collection
#  Note: I use [::2] to skip every other image because of  
#    a quirk of the data I'm using.
arr = io.ImageCollection(ims_f)[::2].concatenate()

ValueError: Image dimensions must agree.

Next, I will select the first channel (the red channel in `RGB` format).

In [20]:
arr = arr[:, :, :, 0]

Now, let's save this as a TIF stack, or multi-image TIF.

In [21]:
io.imsave(os.path.join(data_dir, vid, vid+"_fRFP.tiff"), arr)

C:\Users\Pranav\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: E:/transceiver_project/20200715_time_lapse_propagation/time_lapse_20x/0test\0test_fRFP.tiff is a low contrast image
  """Entry point for launching an IPython kernel.


Now let's perform the TIFF stack conversion for multiple videos. First, I'll extract the folder names for each video.

In [5]:
vids = [os.path.split(f)[-1] for f in glob(data_dir + "*_?")]
print(*vids, sep=", ")

10000_A, 10000_B, 10000_C, 1000_A, 1000_B, 1000_C, 160000_A, 160000_B, 160000_C, 20000_A, 20000_B, 20000_C, 2500_A, 2500_B, 2500_C, 40000_A, 40000_B, 40000_C, 5000_A, 5000_B, 5000_C, 80000_A, 80000_B, 80000_C


Now, we simply perform the same steps as before, for each video. We'll add a progress bar, since this could take a while.

In [23]:
iterator = vids
iterator = tqdm(iterator)
for vid in iterator:
    # Read images
    ims_f = glob(os.path.join(data_dir, vid, "*/*CH3.tif"))

    # Convert to image collection
    arr = io.ImageCollection(ims_f)[::2].concatenate()
    
    # Select red channel of RGB
    arr = arr[:, :, :, 0]
    
    # Save as TIFF stack
    io.imsave(os.path.join(data_dir, vid, vid+"_fRFP.tiff"), arr)

100%|██████████| 24/24 [10:13<00:00, 25.54s/it]


<hr>

## Cropping TIFF stacks

In [22]:
vid = vids[0]

# Read in first TIFF
im_f = glob(os.path.join(data_dir, vid, "*_fRFP.tiff"))[0]
im = io.imread(im_f)

# Get shape for cropping
n_im, im_ny, im_nx = im.shape
crop_start = (im_nx - im_ny)//2

In [23]:
iterator = vids
iterator = tqdm(iterator)
for vid in iterator:
    # Read 
    im_f = glob(os.path.join(data_dir, vid, "*_fRFP.tiff"))[0]
    im = io.imread(im_f)

    # Crop
    im_crop = im[:, :, crop_start:(im_ny+crop_start)]

    # Save
    io.imsave(os.path.join(data_dir, vid, vid+"_fRFP_cropped.tiff"), im_crop)

100%|██████████| 24/24 [05:21<00:00, 13.39s/it]
